In [11]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from sklearn.model_selection import KFold
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

## Part-1

In [2]:
#transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#load the dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

#trainset into 80% training and 20% validation
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_subset, val_subset = random_split(trainset, [train_size, val_size])

#define batch_size
batch_size = 16

# data loaders for train validate and test
trainloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
valloader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

#define classes
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
num_classes = 10

# load vgg16 
vgg16 = models.vgg16_bn(pretrained=True)  # Load VGG16 with batch normalization
vgg16.classifier[6] = torch.nn.Linear(vgg16.classifier[6].in_features, num_classes)

# load resnet18
resnet18 = models.resnet18(pretrained=True)  #ResNet18
resnet18.fc = torch.nn.Linear(resnet18.fc.in_features, num_classes)

# load alexnet
alexnet = models.alexnet(pretrained=True)  # Load AlexNet
alexnet.classifier[6] = torch.nn.Linear(alexnet.classifier[6].in_features, num_classes)

#put them as dictionary
models_dict = {
    "VGG16": vgg16,
    "ResNet18": resnet18,
    "AlexNet": alexnet
}

/Users/Teoman/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /Users/Teoman/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100.0%


In [10]:
#train the models
def train_model(model, trainloader, valloader, num_epochs=5, lr=0.001):
    cost = nn.CrossEntropyLoss() #use cross entropy as loss function
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9) #as optimizer using sgd
    start_time = time.time() #start time for calculate the time of the model to train

    model = model.to(device)  # Move model to GPU

    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()  # models training model

        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            optimizer.zero_grad()  #zgradients as zero 
            outputs = model(inputs)  #forward pass
            loss = cost(outputs, labels)  #compute the loss
            loss.backward()  # backward pass
            optimizer.step()  # update the weights

            running_loss += loss.item() #all the loss here

        # validaton through iterations(epochs)
        val_accuracy = evaluate_model(model, valloader)
        print(f"epoch [{epoch+1}/{num_epochs}], loss: {running_loss/len(trainloader):.4f}, validation accuracy: {val_accuracy:.2f}%")
    
    training_time = time.time() - start_time #finish calculate time of model training
    return training_time


## Cross validation for models

In [12]:
#evaluate the models
def evaluate_model(model, dataloader):
    model.eval()  # model evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  
        for inputs, labels in dataloader: #iterate through dataloaders
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0) #calculate the total tp in these lines
            correct += (predicted == labels).sum().item() 

    accuracy = 100 * correct / total
    return accuracy

# calculate model size by parameters
def calculate_model_size_in_memory(model):
    total_params = sum(param.numel() for param in model.parameters())
    return total_params

#cross-validation function
def cross_validate(model_class, model_name, trainset, num_folds=5, num_epochs=5):
    kf = KFold(n_splits=num_folds, shuffle=True)
    #store teh accuracy time and model size in lists
    accuracies = [] 
    training_times = []
    model_sizes = []

    print(f"\n=== Cross-Validation for {model_name} ===") 

    #iterate through
    for fold, (train_idx, val_idx) in enumerate(kf.split(trainset)):
        print(f"\nFold {fold + 1}/{num_folds}")

        # data loaders for train and validaton dataseets
        train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)
        fold_trainloader = DataLoader(trainset, batch_size=batch_size, sampler=train_sampler)
        fold_valloader = DataLoader(trainset, batch_size=batch_size, sampler=val_sampler)

        # Initialize a new model instance for each fold
        model = model_class.to(device)  # Move model to GPU

        # Train the model
        training_time = train_model(model, fold_trainloader, fold_valloader, num_epochs=num_epochs)

        # Evaluate model on the validation set
        accuracy = evaluate_model(model, fold_valloader)
        model_size = calculate_model_size_in_memory(model)

        #put the result to their corresponding lists
        accuracies.append(accuracy)
        training_times.append(training_time)
        model_sizes.append(model_size)

        print(f"Fold {fold + 1}: Accuracy = {accuracy:.2f}%, Time = {training_time:.2f} sec, Total Parameters: {model_size}")

    # calculate teh average of accuracy time and mode size
    avg_accuracy = sum(accuracies) / num_folds
    avg_time = sum(training_times) / num_folds
    avg_size = sum(model_sizes) / num_folds

    # print ht results
    print(f"\nAverage Accuracy for {model_name}: {avg_accuracy:.2f}%")
    print(f"Average Training Time: {avg_time:.2f} sec")
    print(f"Average Model Size (Total Parameters): {avg_size}")

    return avg_accuracy, avg_time, avg_size

In [13]:
# validate the each model
for model_name, model in models_dict.items(): #take it from the models dctionary
    avg_acc, avg_time, avg_size = cross_validate(model, model_name, train_subset, num_folds=5, num_epochs=5)
    



=== Cross-Validation for VGG16 ===

Fold 1/5


KeyboardInterrupt: 

In [ ]:
# see the best model
best_model = max(models_dict, key=lambda name: evaluate_model(models_dict[name], valloader))
print(f"\nBest Model: {best_model}")

# eval the best model on the test dataset   
test_accuracy = evaluate_model(models_dict[best_model], testloader)
print(f"Test Accuracy for Best Model ({best_model}): {test_accuracy:.2f}%")


In [ ]:
#display the confusion matrix on the best model

def plot_confusion_matrix(model, dataloader):
    model.eval()
    all_preds = torch.tensor([], dtype=torch.long)
    all_labels = torch.tensor([], dtype=torch.long)

    # get all the true labels
    with torch.no_grad():
        for inputs, labels in dataloader:#iterate through dataloaders
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds = torch.cat((all_preds, preds), dim=0)
            all_labels = torch.cat((all_labels, labels), dim=0)

    cm = confusion_matrix(all_labels.numpy(), all_preds.numpy())

    # plot the confusion matrix
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix for {best_model}')
    plt.colorbar()

    tick_marks = range(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    for i in range(len(classes)):
        for j in range(len(classes)):
            plt.text(j, i, cm[i, j], horizontalalignment="center", color="black")

    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

# Plot confusion matrix for the best model
plot_confusion_matrix(models_dict[best_model], testloader)

=== Cross-Validation for VGG16 ===

Fold 1/5
epoch [1/5], loss: 1.0351, validation accuracy: 82.47%
epoch [2/5], loss: 0.6056, validation accuracy: 84.84%
epoch [3/5], loss: 0.4402, validation accuracy: 85.44%
epoch [4/5], loss: 0.3368, validation accuracy: 85.28%
epoch [5/5], loss: 0.2618, validation accuracy: 86.05%
Fold 1: Accuracy = 86.05%, Time = 819.11 sec, Total Parameters: 134309962

Fold 2/5
epoch [1/5], loss: 0.2823, validation accuracy: 95.14%
epoch [2/5], loss: 0.2034, validation accuracy: 94.99%
epoch [3/5], loss: 0.1555, validation accuracy: 94.61%
epoch [4/5], loss: 0.1415, validation accuracy: 93.69%
epoch [5/5], loss: 0.1091, validation accuracy: 91.22%
Fold 2: Accuracy = 91.22%, Time = 816.17 sec, Total Parameters: 134309962

Fold 3/5
epoch [1/5], loss: 0.1364, validation accuracy: 98.55%
epoch [2/5], loss: 0.0998, validation accuracy: 98.04%
epoch [3/5], loss: 0.0773, validation accuracy: 96.74%
epoch [4/5], loss: 0.0715, validation accuracy: 97.69%
epoch [5/5], loss: 0.0590, validation accuracy: 96.99%
Fold 3: Accuracy = 96.99%, Time = 827.52 sec, Total Parameters: 134309962

Fold 4/5
epoch [1/5], loss: 0.0724, validation accuracy: 98.70%
epoch [2/5], loss: 0.0576, validation accuracy: 99.10%
epoch [3/5], loss: 0.0433, validation accuracy: 98.97%
epoch [4/5], loss: 0.0468, validation accuracy: 98.46%
epoch [5/5], loss: 0.0440, validation accuracy: 98.65%
Fold 4: Accuracy = 98.65%, Time = 816.69 sec, Total Parameters: 134309962

Fold 5/5
epoch [1/5], loss: 0.0550, validation accuracy: 99.34%
epoch [2/5], loss: 0.0365, validation accuracy: 99.45%
epoch [3/5], loss: 0.0302, validation accuracy: 99.65%
epoch [4/5], loss: 0.0288, validation accuracy: 98.94%
epoch [5/5], loss: 0.0282, validation accuracy: 99.31%
Fold 5: Accuracy = 99.31%, Time = 814.72 sec, Total Parameters: 134309962

Average Accuracy for VGG16: 94.45%
Average Training Time: 818.84 sec
Average Model Size (Total Parameters): 134309962.0

=== Cross-Validation for ResNet18 ===

Fold 1/5
epoch [1/5], loss: 1.5030, validation accuracy: 63.89%
epoch [2/5], loss: 1.1138, validation accuracy: 71.20%
epoch [3/5], loss: 0.9304, validation accuracy: 73.17%
epoch [4/5], loss: 0.8335, validation accuracy: 75.38%
epoch [5/5], loss: 0.7225, validation accuracy: 75.97%
Fold 1: Accuracy = 75.97%, Time = 296.07 sec, Total Parameters: 11181642

Fold 2/5
epoch [1/5], loss: 0.7210, validation accuracy: 82.24%
epoch [2/5], loss: 0.6193, validation accuracy: 81.00%
epoch [3/5], loss: 0.5385, validation accuracy: 81.83%
epoch [4/5], loss: 0.4804, validation accuracy: 82.56%
epoch [5/5], loss: 0.4294, validation accuracy: 80.62%
Fold 2: Accuracy = 80.62%, Time = 293.38 sec, Total Parameters: 11181642

Fold 3/5
epoch [1/5], loss: 0.4578, validation accuracy: 90.24%
epoch [2/5], loss: 0.3932, validation accuracy: 89.70%
epoch [3/5], loss: 0.3562, validation accuracy: 89.25%
epoch [4/5], loss: 0.3263, validation accuracy: 87.95%
epoch [5/5], loss: 0.2666, validation accuracy: 88.14%
Fold 3: Accuracy = 88.14%, Time = 293.90 sec, Total Parameters: 11181642

Fold 4/5
epoch [1/5], loss: 0.3638, validation accuracy: 92.75%
epoch [2/5], loss: 0.2705, validation accuracy: 90.90%
epoch [3/5], loss: 0.2260, validation accuracy: 93.75%
epoch [4/5], loss: 0.1955, validation accuracy: 91.95%
epoch [5/5], loss: 0.1651, validation accuracy: 91.01%
Fold 4: Accuracy = 91.01%, Time = 293.06 sec, Total Parameters: 11181642

Fold 5/5
epoch [1/5], loss: 0.2118, validation accuracy: 96.40%
epoch [2/5], loss: 0.1595, validation accuracy: 94.99%
epoch [3/5], loss: 0.1350, validation accuracy: 93.71%
epoch [4/5], loss: 0.1331, validation accuracy: 95.71%
epoch [5/5], loss: 0.1093, validation accuracy: 95.26%
Fold 5: Accuracy = 95.26%, Time = 291.91 sec, Total Parameters: 11181642

Average Accuracy for ResNet18: 86.20%
Average Training Time: 293.66 sec
Average Model Size (Total Parameters): 11181642.0

=== Cross-Validation for AlexNet ===

Fold 1/5
---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-12-70d0800834a6> in <cell line: 2>()
      1 # validate the each model
      2 for model_name, model in models_dict.items(): #take it from the models dctionary
----> 3     avg_acc, avg_time, avg_size = cross_validate(model, model_name, train_subset, num_folds=5, num_epochs=5)
      4 

12 frames
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py in _max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode, return_indices)
    828     if stride is None:
    829         stride = torch.jit.annotate(List[int], [])
--> 830     return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
    831 
    832 

RuntimeError: Given input size: (256x1x1). Calculated output size: (256x0x0). Output size is too small
